# Linear Regression

1. Prepare Data  
  
2. Designing the model
   <ol type="i">
      <li>input size</li>
      <li>output size</li>
      <li>forward pass</li>
   </ol>  
   
3. Construct loss and optimizer  

4. Training Loop
   <ol type="I">
      <li>forward pass: compute prediction and loss</li>
      <li>backward pass: gradients</li>
      <li>update weights</li>
   </ol>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

### 0. Prepare Data

### 1. Desining the Model

### 2. Loss Function and the Optimizer

### 3. Training Loop

### 4. Plots